<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/5_3_DL_onehot_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [298]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [294]:
!pip install imbalanced-learn

## Deep Learning model with One hot encoding data
1. import new return label data.
2. Under sampling data to address imbalance class issue.
4. Train model
5. Eval model
6. Create Risk-heatmap

In [299]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import tensorflow as tf
import random
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [219]:
def reset_seeds():
    '''
    reset random seeds for modeling
    '''
    np.random.seed(1)
    random.seed(2)
    if tf.__version__[0] == '2':
        tf.random.set_seed(3)
    else:
        tf.set_random_seed(3)
    print("RANDOM SEEDS RESET")

In [5]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [6]:
df = df.drop(columns='Unnamed: 0')

In [7]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
dtype: int64

## import return label data

In [8]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [9]:
## import return new labeled data
file = path + 'return.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [10]:
len(df_fraud2)

24

In [11]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [12]:
df = label_fraud(df, df_fraud,df_fraud2)

In [13]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Encoding

In [317]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
fraud                3
dtype: int64

In [38]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [248]:
## remove some fields with many classes
cols1 = ['distributor', 'sales', 'invoice_no', 'product_no']
df_less = df.drop(columns=cols1)

In [249]:
def field_onehotencoding(df):
  cols = [ 'branch', 'inv_type',  'prod_cla', 'cust_type']
  df[cols] = df[cols].astype('str') 
  df_enc = pd.get_dummies(df)
  return df_enc

In [250]:
df_less = field_onehotencoding(df_less)

In [251]:
df_less.shape

(228601, 67)

## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [302]:
df_fraud = df_less[df_less['fraud'] == 1]
df_non_fraud = df_less[df_less['fraud'] == 0]
df_fraud2 = df_less[df_less['fraud'] == 2]

In [311]:
df_fraud2.shape

(433, 67)

In [303]:
df_non_fraud.shape

(228014, 67)

In [307]:
df_non_fraud.nunique()

qty                 202
total_amt         15896
sale_price         8237
ship_qty            202
return                2
                  ...  
cust_type_KX01        2
cust_type_LX01        2
cust_type_MX01        2
cust_type_NX01        2
cust_type_ZZ01        2
Length: 67, dtype: int64

In [312]:
def sampling_record(df_non_fraud,df_fraud,df_fraud2):
  '''
  Random sampling from unlabled records.
  concat with labeled records
  '''
  df = df_non_fraud.sample(frac=1,replace=False)

  sample_size = len(df_non_fraud) - 8000
 
  ## Build training dataset
  df_train = df.iloc[:sample_size,:]
  df_train = pd.concat([df_train,df_fraud2])
  df_train.loc[df_train['fraud'] == 2, 'fraud'] = 1 
  df_train = df_train.sample(frac=1,replace=False)
  print(len(df_train))

  ## Build eval dataset
  df_test = df.iloc[sample_size:,:]
  df_test = pd.concat([df_test,df_fraud])
  df_test = df_test.sort_index()
  print(len(df_test))

  return df_train, df_test


In [313]:
df_train, df_test = sampling_record(df_non_fraud, df_fraud, df_fraud2)

220447
8154


In [314]:
df_train.shape

(220447, 67)

In [316]:
df_test.shape

(8154, 67)

In [315]:
df_train.nunique()

qty                 202
total_amt         15737
sale_price         8375
ship_qty            202
return                2
                  ...  
cust_type_KX01        2
cust_type_LX01        2
cust_type_MX01        2
cust_type_NX01        2
cust_type_ZZ01        2
Length: 67, dtype: int64

SMOTE 

In [318]:
y_train = df_train['fraud'].values
X_train = df_train.drop(columns=['fraud'])

In [319]:
# define pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_train, y_train = pipeline.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [320]:
X_train.shape

(66003, 66)

In [321]:
y_train.sum()

22001.0

In [322]:
y_train.shape

(66003,)

## Standardscale data

In [45]:
scaler = StandardScaler()

In [323]:

X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [324]:
y_test = df_test['fraud'].values
X_test = df_test.drop(columns=['fraud'])

X_test = scaler.transform(X_test)
y_test = y_test.reshape(len(y_test),1)

In [325]:
X_train.shape

(66003, 66)

In [326]:
y_test.shape

(8154, 1)

In [327]:
y_train.sum()

22001.0

## Build model

In [328]:
# define the  model
def build_model():
  model = Sequential()
  model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the keras model
  # opt = tf.keras.optimizers.Adam(clipnorm=1, learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [330]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8576      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 16,897
Trainable params: 16,897
Non-trainable params: 0
_________________________________________________________________


## Training model

In [354]:
## clear previous model.  Make sure training from 0. 
del model
clear_session()
tf.compat.v1.reset_default_graph()
reset_seeds()
model = build_model()

RANDOM SEEDS RESET


In [355]:
model.fit(X_train, y_train, epochs=200,batch_size=64,class_weight={0:1, 1:2})

Epoch 1/200
1032/1032 [==============================] - 3s 2ms/step - loss: 0.2395 - accuracy: 0.9074
Epoch 2/200
1032/1032 [==============================] - 3s 2ms/step - loss: 0.1548 - accuracy: 0.9446
Epoch 3/200
1032/1032 [==============================] - 3s 2ms/step - loss: 0.1320 - accuracy: 0.9536
Epoch 4/200
1032/1032 [==============================] - 3s 3ms/step - loss: 0.1119 - accuracy: 0.9623
Epoch 5/200
1032/1032 [==============================] - 3s 3ms/step - loss: 0.1027 - accuracy: 0.9652
Epoch 6/200
1032/1032 [==============================] - 2s 2ms/step - loss: 0.0931 - accuracy: 0.9688
Epoch 7/200
1032/1032 [==============================] - 3s 3ms/step - loss: 0.0862 - accuracy: 0.9704
Epoch 8/200
1032/1032 [==============================] - 3s 2ms/step - loss: 0.0808 - accuracy: 0.9732
Epoch 9/200
1032/1032 [==============================] - 3s 3ms/step - loss: 0.0799 - accuracy: 0.9739
Epoch 10/200
1032/1032 [==============================] - 3s 3ms/step - l

In [356]:
loss,accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

255/255 [==============================] - 0s 1ms/step - loss: 3.1830 - accuracy: 0.9788
3.183027744293213 0.9787834286689758


## Predict on eval dataset for further labeling.

In [367]:
y_pred = model.predict(X_test)
y_cls = (y_pred > 0.5).astype("int32")

In [358]:
(y_pred > 0.4).sum()

104

In [334]:
def eval_model(y_test, y_cls):
  y1 = y_test.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))
  print(confusion_matrix(y1,y2))


In [284]:
eval_model(y_train, y_cls)

0.5355162712576108
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     22000
         1.0       0.09      0.09      0.09       433

    accuracy                           0.97     22433
   macro avg       0.54      0.54      0.54     22433
weighted avg       0.96      0.97      0.97     22433

[[21632   368]
 [  395    38]]


In [203]:
eval_model(y_train, y_cls)

0.5064665001502765
              precision    recall  f1-score   support

         0.0       0.99      0.67      0.80     36799
         1.0       0.01      0.35      0.02       433

    accuracy                           0.66     37232
   macro avg       0.50      0.51      0.41     37232
weighted avg       0.98      0.66      0.79     37232

[[24527 12272]
 [  283   150]]


In [290]:
eval_model(y_test, y_cls)

0.6254853896103897
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      8000
         1.0       0.16      0.28      0.20       154

    accuracy                           0.96      8154
   macro avg       0.57      0.63      0.59      8154
weighted avg       0.97      0.96      0.96      8154

[[7774  226]
 [ 111   43]]


In [368]:
eval_model(y_test, y_cls)

0.5975243506493507
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      8000
         1.0       0.38      0.20      0.26       154

    accuracy                           0.98      8154
   macro avg       0.68      0.60      0.63      8154
weighted avg       0.97      0.98      0.98      8154

[[7950   50]
 [ 123   31]]


In [362]:
def export_result(df_test,y_cls, y_pred):
  '''
  Attach the predicte result (probability) into original records.
  Export to a csv file for further investication
  '''
  df_test['predict'] = y_cls
  df_test['prob']  = y_pred
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'pred_result.csv'
  df_test.to_csv(file_csv)
  return True


In [364]:
export_result(df_test,y_cls,y_pred)

True